In [4]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load clean data
df = pd.read_excel("sales.xlsx")

# Drop unnecessary unnamed columns and clean headers
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df.columns = df.columns.str.strip()

# Keep only rows with valid 'Financial Period' format
df['Financial Period'] = df['Financial Period'].astype(str).str.strip()
df = df[df['Financial Period'].str.match(r'QTR \d \d{4}', na=False)]

# Convert 'QTR x YYYY' to datetime
df['Date'] = df['Financial Period'].str.extract(r'QTR (\d) (\d{4})').apply(lambda x: f"{x[1]}Q{x[0]}", axis=1)
df['Date'] = pd.PeriodIndex(df['Date'], freq='Q').to_timestamp(how='end')

# Set datetime index
df = df.sort_values('Date')
df = df.set_index('Date')
df = df.asfreq('QE')  # updated frequency
df = df.ffill()       # fill forward missing data

# Forecast for each product
future_sales = {}

for column in df.columns.drop('Financial Period', errors='ignore'):
    series = df[column]

    if len(series) >= 8 and not series.isnull().any():
        model = ExponentialSmoothing(series, trend="add", seasonal="add", seasonal_periods=4)
        fitted = model.fit()
        forecast = fitted.forecast(4)
        # Round and replace negatives with 0
        forecast = forecast.round().astype(int).clip(lower=0)
        future_sales[column] = forecast
    else:
        print(f"Skipping {column}: insufficient or invalid data")

# Create forecast DataFrame
forecast_index = pd.date_range(start=df.index[-1] + pd.offsets.QuarterEnd(), periods=4, freq='QE')
forecast_df = pd.DataFrame(future_sales, index=forecast_index)

# Format index like 'QTR X YYYY'
forecast_df.index = ['QTR {} {}'.format(d.quarter, d.year) for d in forecast_df.index]

# Output results
print("\n Final Sales Forecast for 2025:\n")
print(forecast_df)

# Save to Excel
forecast_df.to_excel("sales_forecast_2025.xlsx")



✅ Final Sales Forecast for 2025:

            Bulbs B2B  Bulbs B2C  LED B2B  LED B2C  Switch Fans B2B  \
QTR 1 2025         12          8      120       83                9   
QTR 2 2025          9          6      133       82                7   
QTR 3 2025         10          4      134       95                8   
QTR 4 2025          9          6      141       99                7   

            Switch Fans B2C  Remote Fans B2B  Remote Fans B2C  \
QTR 1 2025                6              116               54   
QTR 2 2025                6              116               63   
QTR 3 2025                5              121               60   
QTR 4 2025                4              127               62   

            Manual Switches B2B  Manual Switches B2C  \
QTR 1 2025                    0                    0   
QTR 2 2025                    0                    0   
QTR 3 2025                    0                    0   
QTR 4 2025                    0                    0   

  

In [8]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

# Load and clean data
df = pd.read_excel("sales.xlsx")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df.columns = df.columns.str.strip()
df['Financial Period'] = df['Financial Period'].astype(str).str.strip()
df = df[df['Financial Period'].str.match(r'QTR \d \d{4}', na=False)]

# Convert 'QTR x YYYY' to datetime
df['Date'] = df['Financial Period'].str.extract(r'QTR (\d) (\d{4})').apply(lambda x: f"{x[1]}Q{x[0]}", axis=1)
df['Date'] = pd.PeriodIndex(df['Date'], freq='Q').to_timestamp(how='end')
df = df.sort_values('Date').set_index('Date')
df = df.asfreq('QE').ffill()

# Error metric storage
error_results = []

# Forecast for each product
for column in df.columns.drop('Financial Period', errors='ignore'):
    series = df[column]

    # Separate training (up to 2023) and actual 2024
    train = series[series.index.year <= 2023]
    test = series[series.index.year == 2024]

    if len(train) >= 8 and len(test) == 4 and not train.isnull().any():
        model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=4)
        fitted = model.fit()
        forecast = fitted.forecast(4).clip(lower=0)

        # Error metrics
        rmse = np.sqrt(mean_squared_error(test, forecast))
        error_results.append({
            "Product": column,
            "RMSE": round(rmse, 2)
        })
    else:
        print(f"Skipping {column}: insufficient or missing data")

# Output error metrics
error_df = pd.DataFrame(error_results)
print("\n Forecast Accuracy for 2024:\n")
print(error_df)
print(f"\nAverage RMSE: {error_df['RMSE'].mean():.2f}")


# Save to Excel if needed
error_df.to_excel("forecast_error_2024.xlsx", index=False)



 Forecast Accuracy for 2024:

                     Product   RMSE
0                  Bulbs B2B   4.78
1                  Bulbs B2C   4.63
2                    LED B2B  17.77
3                    LED B2C  14.80
4            Switch Fans B2B   3.64
5            Switch Fans B2C   2.25
6            Remote Fans B2B  12.32
7            Remote Fans B2C   8.07
8        Manual Switches B2B   5.68
9        Manual Switches B2C   1.55
10  Smart Touch Switches B2B  10.90
11  Smart Touch Switches B2C   5.22
12             Doorbells B2B   7.97
13             Doorbells B2C   7.39
14       Video Doorbells B2B   4.02
15       Video Doorbells B2C   5.51

Average RMSE: 7.28
